In [16]:
from matplotlib.pyplot import draw, fill
import pyrender
import trimesh
from trimesh import remesh
import pandas as pd
import numpy as np
from trimesh import Trimesh
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
import pickle
import matplotlib.mlab as mlab
import seaborn as sns
import skeletor as sk
from view_data import view_mesh
import copy
from sklearn.preprocessing import MinMaxScaler
from project_statistics import get_outliers
from create_dataset import load_dataset,dir_to_sorted_file_list
from trimesh.repair import fill_holes
from trimesh.points import PointCloud
import ast
import random
from scipy.stats import wasserstein_distance
from random import choice
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler

In [9]:
with open('normalized_scalar_features.pkl', 'rb') as f:
    normalized_data = pickle.load(f)

In [23]:
with open('normalized_features_classes.pkl', 'rb') as f:
    data = pickle.load(f)

In [45]:
normalized_data

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.039329,-0.116012,0.230250,1.0,-0.109650,-0.026311,"[0.0015568240788790867, 0.011416709911779968, ...","[0.23664198719878007, 0.22873653163185456, 0.1...","[0.04733241036136359, 0.12355785629727724, 0.1...","[0.09370171953690884, 0.19420534119865968, 0.2...","[0.20435803286582796, 0.15628323200706273, 0.1...","[0.13328919119565, 0.28198800136439334, 0.2750..."
1,1,-0.640930,-0.415322,-0.118042,0.272779,1.0,0.097077,-0.026489,"[0.04924623115577889, 0.054773869346733665, 0....","[0.24772371746019012, 0.22417873330391883, 0.1...","[0.12446351931330472, 0.25626729774176726, 0.1...","[0.151618466968834, 0.300790180899282, 0.27189...","[0.3746741005174281, 0.28757370342144317, 0.16...","[0.07649111547872127, 0.2777064698568048, 0.32..."
2,2,-0.603237,-0.095673,-0.118419,0.280905,1.0,-0.043228,-0.026263,"[0.015625, 0.03780241935483871, 0.137096774193...","[0.25301446542142325, 0.23644242922777522, 0.1...","[0.15300042132295408, 0.21009971309913128, 0.1...","[0.14629938005336757, 0.26599522500652045, 0.2...","[0.29506650883775054, 0.24288265152579097, 0.1...","[0.11953534097064783, 0.3514635956904681, 0.29..."
3,3,-0.520618,0.392970,-0.119893,0.313312,1.0,-0.468747,-0.026398,"[0.011866235167206042, 0.02481121898597627, 0....","[0.20333848283610537, 0.23802740605501274, 0.1...","[0.05069919547378769, 0.09237004193166543, 0.0...","[0.061753907268824106, 0.1599622815641113, 0.1...","[0.1285636899865578, 0.1484260578215597, 0.138...","[0.08151596011476035, 0.2712316674357482, 0.31..."
4,4,-0.724641,-0.475575,-0.095315,-0.114249,1.0,-0.306213,-0.026249,"[0.06380952380952382, 0.10952380952380952, 0.2...","[0.2399117529081428, 0.2335138387484958, 0.155...","[0.17721620537505015, 0.18864821500200563, 0.2...","[0.1078018451664661, 0.2600882470918572, 0.267...","[0.2778780585639791, 0.25130365022061774, 0.16...","[0.07882069795427196, 0.2884476534296029, 0.31..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,-0.365028,-0.011320,-0.113034,0.171236,1.0,-0.081913,-0.026487,"[0.08372530573847602, 0.009877704609595485, 0....","[0.1711191335740072, 0.23331327717609307, 0.20...","[0.10609707180104291, 0.027476935419173686, 0....","[0.10146409947853992, 0.1478740473325311, 0.25...","[0.1829121540312876, 0.11217408744484557, 0.09...","[0.07059767348576013, 0.20198555956678702, 0.2..."
1810,1810,-0.089583,0.813581,-0.138504,0.844492,1.0,-0.125955,-0.026451,"[0.0903030303030303, 0.15575757575757576, 0.38...","[0.16446496687412165, 0.24053402931138326, 0.2...","[0.18598674964866493, 0.2187111021883156, 0.21...","[0.07153182091949407, 0.1783778357759486, 0.24...","[0.13278458140935556, 0.1505320216823931, 0.14...","[0.08239309375627384, 0.27639028307568764, 0.3..."
1811,1811,-0.509943,-0.716180,-0.046532,-0.572259,1.0,-0.403390,-0.026294,"[0.008452192287374538, 0.0306391970417327, 0.0...","[0.2840779153042187, 0.1652189612630143, 0.140...","[0.1258199763285121, 0.4619952255812554, 0.094...","[0.25559188750025075, 0.2389616642259624, 0.12...","[0.4473911211859817, 0.21958314108607996, 0.13...","[0.10658187726935345, 0.2844189452145479, 0.28..."
1812,1812,0.413064,1.070660,-0.142995,1.019764,1.0,-0.306231,-0.026260,"[0.1552892131318395, 0.26003126628452317, 0.28...","[0.23453768531707023, 0.2171845848295785, 0.16...","[0.06275202118482556, 0.09860172126707725, 0.1...","[0.15445262503259977, 0.1181415130298714, 0.18...","[0.29749032038036394, 0.104580014845427, 0.078...","[0.2087588019339178, 0.29097036933015025, 0.20..."


In [12]:
with open('example_700.pkl', 'rb') as f:
    query = pickle.load(f)

In [13]:
sampled_min_max = np.load('sampeld_min_max.npy')
sampled_mean_std = np.load('sampeld_mean_std.npy')

In [14]:
def standardize(data, idx):
    data_mean = sampled_mean_std[idx][0]  
    data_std = sampled_mean_std[idx][1]  
    normalized_descriptor_data = (data - data_mean)/data_std
    return normalized_descriptor_data

In [104]:
def predict(normalized_data, query, sampled_min_max, sampled_mean_std, n):
    comaprsion_df = normalized_data.copy()
    scalar_query_features = np.array(query[1:8], dtype = 'float32')
    descriptors_query_features = np.vstack(np.array(query[8:]))
    comaprsion_values = {}
    for index, row in normalized_data.iterrows():
        scalar_features = np.array(row[1:8], dtype = 'float32')
        descriptor_features = np.vstack(np.array(row[8:]))
        scalar_distance = distance.cosine(scalar_query_features,scalar_features)
        total_descriptor_distance = 0
        for i in range(len(descriptors_query_features)):
            descriptor_query_feature = descriptors_query_features[i]
            descriptor_feature = descriptor_features[i]
            descriptor_distance = wasserstein_distance(descriptor_query_feature,descriptor_feature)

            descriptor_distance_scaled = (descriptor_distance - sampled_min_max[i][0]) / (sampled_min_max[i][1] - sampled_min_max[i][0])
            total_descriptor_distance += descriptor_distance
            total_descriptor_distance = total_descriptor_distance
        total_distance = scalar_distance + total_descriptor_distance

        if np.isnan(total_distance ): 
            comaprsion_values[index] = 999
        else:
            comaprsion_values[index] = total_distance
    
    extract = lambda x: x[0]
    pred = list(map(extract,sorted(comaprsion_values.items(), key=lambda x: x[1])[:n]))
    return pred

In [109]:
# confusion matrix scaled for number of instances in a class
d = np.zeros((19,19))
class_acc = {}
K = 5
TP = {}
FP = {}
FN = {}
for cls in range(0,19):
    print("Class", cls)
    class_acc[cls] = 0
    TP[cls] = 0
    FP[cls] = 0
    FN[cls] = 0
    
    query_classes = data[data['class']==cls].iloc[:,:14]
    num_rows = 0
    for index, row in query_classes.iterrows():
        print(index, end='\r')
        num_rows +=1
        query_instance = row
        pred = predict(normalized_data, query_instance, sampled_min_max, sampled_mean_std, K)

        pred_list = (map(lambda x: data.loc[x]["class"], pred))
        pred_list = list(map(int, pred_list))
        target_list = np.repeat(cls, K)
        
        
        d[cls] += [pred_list.count(j) for j in range(0,19)]
    
    class_acc[cls] = d[cls][cls]/np.sum(d[cls])
    TP[cls] = d[cls][cls]
    FP[cls] = np.sum(d[cls]) - TP[cls]

for i in range(0,19):
    FN[i] = np.sum(d[:,i]) - TP[i]

Class 0
Class 1
Class 2
Class 3
Class 4
Class 5
Class 6
Class 7
Class 8
Class 9
Class 10
Class 11
Class 12
Class 13
Class 14
Class 15
Class 16
Class 17
Class 18


In [110]:
P = {}
R = {}
for i in range(0,19):
    P[i] = 0
    R[i] = 0
    P[i] = TP[i]/(TP[i] + FP[i])
    R[i] = TP[i]/(TP[i] + FN[i])

In [111]:
class_acc

{0: 0.41,
 1: 0.642,
 2: 0.518,
 3: 0.542,
 4: 0.38,
 5: 0.366,
 6: 0.424,
 7: 0.33535353535353535,
 8: 0.456,
 9: 0.42,
 10: 0.638,
 11: 0.432,
 12: 0.576,
 13: 0.37,
 14: 0.436,
 15: 0.646,
 16: 0.41,
 17: 0.384,
 18: 0.21333333333333335}

In [112]:
global_acc = np.mean(list(class_acc.values()))
global_acc

0.4525624667729931

In [113]:
P

{0: 0.41,
 1: 0.642,
 2: 0.518,
 3: 0.542,
 4: 0.38,
 5: 0.366,
 6: 0.424,
 7: 0.33535353535353535,
 8: 0.456,
 9: 0.42,
 10: 0.638,
 11: 0.432,
 12: 0.576,
 13: 0.37,
 14: 0.436,
 15: 0.646,
 16: 0.41,
 17: 0.384,
 18: 0.21333333333333335}

In [114]:
global_p = np.mean(list(P.values()))
global_p

0.4525624667729931

In [115]:
R

{0: 0.47235023041474655,
 1: 0.5459183673469388,
 2: 0.43456375838926176,
 3: 0.4608843537414966,
 4: 0.42792792792792794,
 5: 0.41496598639455784,
 6: 0.48847926267281105,
 7: 0.3952380952380952,
 8: 0.49032258064516127,
 9: 0.39697542533081287,
 10: 0.6590909090909091,
 11: 0.4098671726755218,
 12: 0.5070422535211268,
 13: 0.39029535864978904,
 14: 0.4105461393596987,
 15: 0.5597920277296361,
 16: 0.40433925049309666,
 17: 0.47761194029850745,
 18: 0.26229508196721313}

In [116]:
global_r = np.mean(list(R.values()))
global_r

0.4530792695730163

In [117]:
TP

{0: 205.0,
 1: 321.0,
 2: 259.0,
 3: 271.0,
 4: 190.0,
 5: 183.0,
 6: 212.0,
 7: 166.0,
 8: 228.0,
 9: 210.0,
 10: 319.0,
 11: 216.0,
 12: 288.0,
 13: 185.0,
 14: 218.0,
 15: 323.0,
 16: 205.0,
 17: 192.0,
 18: 16.0}

In [118]:
FP

{0: 295.0,
 1: 179.0,
 2: 241.0,
 3: 229.0,
 4: 310.0,
 5: 317.0,
 6: 288.0,
 7: 329.0,
 8: 272.0,
 9: 290.0,
 10: 181.0,
 11: 284.0,
 12: 212.0,
 13: 315.0,
 14: 282.0,
 15: 177.0,
 16: 295.0,
 17: 308.0,
 18: 59.0}

In [119]:
FN

{0: 229.0,
 1: 267.0,
 2: 337.0,
 3: 317.0,
 4: 254.0,
 5: 258.0,
 6: 222.0,
 7: 254.0,
 8: 237.0,
 9: 319.0,
 10: 165.0,
 11: 311.0,
 12: 280.0,
 13: 289.0,
 14: 313.0,
 15: 254.0,
 16: 302.0,
 17: 210.0,
 18: 45.0}